In [6]:
import os
from typing import Annotated
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_core.tools import FunctionTool

In [7]:
import random   


def get_random_destination() -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        available_destinations = destinations.copy()

        # Select a random destination
        destination = random.choice(available_destinations)
        return destination

In [8]:
get_random_destination = FunctionTool(
    get_random_destination, description="Returns a random vacation destination."
)

In [11]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

agent = AssistantAgent(
    name=AGENT_NAME,
    model_client=client,
    tools=[get_random_destination],
    system_message=AGENT_INSTRUCTIONS,
    reflect_on_tool_use=True,
)

In [14]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Plan me random day trip.", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='TravelAgent', models_usage=RequestUsage(prompt_tokens=749, completion_tokens=12), content=[FunctionCall(id='call_qJO6whsnOFDXHrjT4yOD3wnl', arguments='{}', name='get_random_destination')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='TravelAgent', models_usage=None, content=[FunctionExecutionResult(content='Paris, France', call_id='call_qJO6whsnOFDXHrjT4yOD3wnl')], type='ToolCallExecutionEvent')]
source='TravelAgent' models_usage=RequestUsage(prompt_tokens=745, completion_tokens=475) content="How about a day trip to the beautiful city of Paris, France? Here’s a suggested itinerary to make the most of your time:\n\n### Morning\n1. **Breakfast at a Café**\n   - Start your day with a classic Parisian breakfast of croissants and café au lait at a local café.\n\n2. **Visit the Eiffel Tower**\n   - Head to the iconic Eiffel Tower. You can either go up for breathtaking views of Paris or enjoy the sights from the Champ de Mars park nearby.\n\n### L